In [58]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [4]:

titan=pd.read_csv('./titanic.txt')
titan.info()
print(titan.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   row.names  1313 non-null   int64  
 1   pclass     1313 non-null   object 
 2   survived   1313 non-null   int64  
 3   name       1313 non-null   object 
 4   age        633 non-null    float64
 5   embarked   821 non-null    object 
 6   home.dest  754 non-null    object 
 7   room       77 non-null     object 
 8   ticket     69 non-null     object 
 9   boat       347 non-null    object 
 10  sex        1313 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 113.0+ KB
   row.names pclass  survived                                 name   age  \
0          1    1st         1         Allen, Miss Elisabeth Walton  29.0   
1          2    1st         0          Allison, Miss Helen Loraine   2.0   
2          3    1st         0  Allison, Mr Hudson Joshua Creighton  30.0   

    

In [6]:
x=titan[['pclass','age','sex']]
y=titan.survived
print(x.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  1313 non-null   object 
 1   age     633 non-null    float64
 2   sex     1313 non-null   object 
dtypes: float64(1), object(2)
memory usage: 30.9+ KB
None


In [8]:
x.age.fillna(x.age.mean(),inplace=True)
print(x.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  1313 non-null   object 
 1   age     1313 non-null   float64
 2   sex     1313 non-null   object 
dtypes: float64(1), object(2)
memory usage: 30.9+ KB
None


C:\Users\gdmmx\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [48]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=1)
dict=DictVectorizer(sparse=False)
# 转成字典形式才能fit_transform
x_train=dict.fit_transform(x_train.to_dict(orient='records'))
print(type(x_train))
print(dict.get_feature_names())

<class 'numpy.ndarray'>
['age', 'pclass=1st', 'pclass=2nd', 'pclass=3rd', 'sex=female', 'sex=male']


In [49]:
dec=DecisionTreeClassifier()
dec.fit(x_train,y_train)
x_test=dict.transform(x_test.to_dict(orient='records'))
print('accuracy:',dec.score(x_test,y_test))

accuracy: 0.8115501519756839


In [54]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=4)
dict = DictVectorizer(sparse=False)
x_train = dict.fit_transform(x_train.to_dict(orient="records"))
x_test = dict.transform(x_test.to_dict(orient="records"))
dec1=DecisionTreeClassifier(max_depth=15,min_impurity_decrease=0.01)
dec1.fit(x_train,y_train)
print("accuracy",dec1.score(x_test,y_test))


accuracy 0.8206686930091185


In [55]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=4)
dict = DictVectorizer(sparse=False)
x_train = dict.fit_transform(x_train.to_dict(orient="records"))
x_test = dict.transform(x_test.to_dict(orient="records"))
dec2=DecisionTreeClassifier(max_depth=10,min_impurity_decrease=0.01)
dec2.fit(x_train,y_train)
print("accuracy",dec2.score(x_test,y_test))

accuracy 0.8206686930091185


In [60]:
rf=RandomForestClassifier(n_jobs=-1)
param={'n_estimators':[1500,2000,4000],'max_depth':[2,3,5,8,12,20]}
gc=GridSearchCV(rf,param_grid=param,cv=3)
gc.fit(x_train,y_train)
print('accuracy',gc.score(x_test,y_test))
print('para model',gc.best_params_)
print('best model',gc.best_estimator_)
print('every result',gc.cv_results_)

accuracy 0.8328267477203647
para model {'max_depth': 3, 'n_estimators': 1500}
best model RandomForestClassifier(max_depth=3, n_estimators=1500, n_jobs=-1)
every result {'mean_fit_time': array([6.09549109, 3.73525508, 4.92038623, 1.74031417, 2.48980713,
       4.26293381, 1.87409576, 2.15475488, 4.02112063, 2.3887639 ,
       3.04691601, 4.83466427, 2.20738029, 2.64854344, 4.95474044,
       1.79287815, 2.54484129, 6.60722812]), 'std_fit_time': array([5.47867849, 1.59991306, 1.10465888, 0.28325251, 0.31986027,
       0.35014919, 0.20083428, 0.09085672, 0.10972059, 0.28945925,
       0.35512852, 0.44032944, 0.38659642, 0.29282955, 0.45142555,
       0.09595359, 0.13121057, 1.67752734]), 'mean_score_time': array([0.48117765, 1.02478886, 1.32907351, 0.52979994, 0.66466864,
       1.27120121, 0.50493932, 0.53272446, 0.89000702, 0.52064164,
       0.68658511, 1.0748361 , 0.53676152, 0.61013269, 1.21966012,
       0.51166495, 0.626127  , 1.7003626 ]), 'std_score_time': array([0.11778341, 0.32

In [64]:
from sklearn.datasets import load_boston
import joblib
from sklearn.linear_model import LinearRegression,SGDRegressor,Ridge,LogisticRegression,Lasso

In [66]:
lb=load_boston()
x_train,x_test,y_train,y_test=train_test_split(lb.data,lb.target,test_size=0.25,random_state=1)
print(x_train.shape)

(379, 13)


In [68]:
from sklearn.preprocessing import StandardScaler

In [70]:
std_x=StandardScaler()
x_train=std_x.fit_transform(x_train)
x_test=std_x.transform(x_test)
std_y=StandardScaler()
y_train=std_y.fit_transform(y_train.reshape(-1,1))
y_test=std_y.transform(y_test.reshape(-1,1))

In [73]:
lr=LinearRegression()
lr.fit(x_train,y_train)
print('系数',lr.coef_)

系数 [[-0.12026411  0.15044778  0.02951803  0.07470354 -0.28043353  0.22170939
   0.02190624 -0.35275513  0.29939558 -0.2028089  -0.23911894  0.06305081
  -0.45259462]]


In [78]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, LogisticRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, classification_report, roc_auc_score
import joblib
import pandas as pd
import numpy as np

In [79]:
y_predict=lr.predict(x_test)
y_lr_predict=std_y.inverse_transform(y_predict)
joblib.dump(lr,'./house.pkl')
print('predict',y_lr_predict)
print('error',mean_squared_error(y_lr_predict,y_test))

predict [[ 1.12620955]
 [ 0.62994234]
 [-0.47955756]
 [-0.08002168]
 [-0.38323459]
 [-0.26734514]
 [ 1.11558027]
 [-0.48011678]
 [ 0.26773583]
 [ 0.50610896]
 [ 0.54872518]
 [ 0.69878929]
 [-0.12984488]
 [ 0.51624959]
 [ 0.11609798]
 [-0.16307075]
 [-0.58671359]
 [ 1.72804157]
 [ 0.91761907]
 [-1.56015899]
 [-0.16601029]
 [-0.68746111]
 [ 0.31332585]
 [ 0.27297733]
 [ 1.01697482]
 [-1.27028638]
 [-0.95672557]
 [-0.62211389]
 [ 1.5267197 ]
 [-0.8563123 ]
 [-0.12405138]
 [-0.91970532]
 [ 2.28757241]
 [-0.50574043]
 [-0.05595243]
 [-0.21806897]
 [-0.54345359]
 [ 0.52264682]
 [-1.40720286]
 [-0.26284251]
 [ 0.21619076]
 [-0.14338071]
 [ 0.79988591]
 [-0.65772411]
 [-0.33180076]
 [-0.87514574]
 [ 1.91418761]
 [-0.47664284]
 [ 0.43517699]
 [-0.1950607 ]
 [ 0.30927175]
 [ 0.24009869]
 [ 0.30063331]
 [ 0.50569088]
 [-1.94512422]
 [ 0.20018782]
 [-1.30384514]
 [ 0.50366068]
 [-0.6220835 ]
 [ 1.47453167]
 [-0.31823582]
 [ 0.57109939]
 [-0.64702253]
 [-0.35840699]
 [-1.27347275]
 [ 1.08939349]
 [

In [81]:
model=joblib.load('./house.pkl')
y_predict=model.predict(x_test)
print(mean_squared_error(y_test,y_predict))



0.2758842244225055


In [83]:
print(mean_squared_error(std_y.inverse_transform(y_test),std_y.inverse_transform(y_predict)))

0.2758842244225055


In [86]:
sgd=SGDRegressor(eta0=0.008,penalty='l1',alpha=0.005)
sgd.fit(x_train,y_train)
print('系数',sgd.coef_)

系数 [-0.09279447  0.09137297 -0.01475409  0.07827004 -0.20301786  0.25350773
  0.         -0.26686679  0.11503441 -0.03356268 -0.22573739  0.06307533
 -0.4367467 ]


C:\Users\gdmmx\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [88]:
y_sgd_predict=std_y.inverse_transform(sgd.predict(x_test).reshape(-1,1))
y_predict=sgd.predict(x_test)
print('error',mean_squared_error(y_test,y_predict))
print('error before',mean_squared_error(std_y.inverse_transform(y_test),y_sgd_predict))

error 0.2799981437649605
error before 0.2799981437649605
